<a href="https://colab.research.google.com/github/Sujoy-004/smart-city-hybrid-ml/blob/main/notebooks/01_supervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cell 1: Import libraries and load raw data

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options for better data exploration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Cell 1: Loading raw data...")
print("-" * 50)

# Load the raw Delhi AQI dataset
df_raw = pd.read_csv('https://raw.githubusercontent.com/Sujoy-004/smart-city-hybrid-ml/refs/heads/main/data/raw/delhi_aqi.csv')
print(f"✅ Successfully loaded data from delhi_aqi.csv")
print(f"📊 Dataset shape: {df_raw.shape} (rows, columns)")
df_raw.head()

Cell 1: Loading raw data...
--------------------------------------------------
✅ Successfully loaded data from delhi_aqi.csv
📊 Dataset shape: (1461, 12) (rows, columns)


Date  Month  Year  Holidays_Count  Days   PM2.5    PM10     NO2    SO2  \
0     1      1  2021               0     5  408.80  442.42  160.61  12.95   
1     2      1  2021               0     6  404.04  561.95   52.85   5.18   
2     3      1  2021               1     7  225.07  239.04  170.95  10.93   
3     4      1  2021               0     1   89.55  132.08  153.98  10.42   
4     5      1  2021               0     2   54.06   55.54  122.66   9.70   

     CO  Ozone  AQI  
0  2.77  43.19  462  
1  2.60  16.43  482  
2  1.40  44.29  263  
3  1.01  49.19  207  
4  0.64  48.88  149

# Cell 2: Data types and missing value analysis

In [24]:
# 1. Dataset Overview
print(f"\n1️⃣ DATASET OVERVIEW:")
print(f"   • Shape: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")
print(f"   • Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   • Date range: Day {df_raw['Date'].min()} to Day {df_raw['Date'].max()}")
print(f"   • Years covered: {df_raw['Year'].unique()}")


1️⃣ DATASET OVERVIEW:
   • Shape: 1,461 rows × 12 columns
   • Memory usage: 0.13 MB
   • Date range: Day 1 to Day 31
   • Years covered: [2021 2022 2023 2024]


In [30]:
# 2. Data Types Analysis
print(f"\n2️⃣ DATA TYPES:")
print("-" * 40)
dtype_summary = df_raw.dtypes.to_frame('Data_Type')
dtype_summary['Count'] = df_raw.count()
dtype_summary['Missing'] = df_raw.isnull().sum()
dtype_summary['Missing_%'] = (df_raw.isnull().sum() / len(df_raw) * 100).round(2)
dtype_summary['Unique_Values'] = df_raw.nunique()

# Add sample values for better understanding
dtype_summary['Sample_Values'] = ''
for col in df_raw.columns:
    if df_raw[col].dtype in ['object', 'category']:
        sample_vals = df_raw[col].dropna().unique()[:3]
        dtype_summary.loc[col, 'Sample_Values'] = str(list(sample_vals))
    else:
        sample_vals = df_raw[col].dropna().iloc[:3].values
        dtype_summary.loc[col, 'Sample_Values'] = str(list(sample_vals))

print(dtype_summary.to_string())


2️⃣ DATA TYPES:
----------------------------------------
               Data_Type  Count  Missing  Missing_%  Unique_Values                                                 Sample_Values
Date               int64   1461        0        0.0             31                       [np.int64(1), np.int64(2), np.int64(3)]
Month              int64   1461        0        0.0             12                       [np.int64(1), np.int64(1), np.int64(1)]
Year               int64   1461        0        0.0              4              [np.int64(2021), np.int64(2021), np.int64(2021)]
Holidays_Count     int64   1461        0        0.0              2                       [np.int64(0), np.int64(0), np.int64(1)]
Days               int64   1461        0        0.0              7                       [np.int64(5), np.int64(6), np.int64(7)]
PM2.5            float64   1461        0        0.0           1391   [np.float64(408.8), np.float64(404.04), np.float64(225.07)]
PM10             float64   1461        

In [27]:
# 3. Missing Values Summary
print(f"\n3️⃣ MISSING VALUES SUMMARY:")
print("-" * 40)
missing_summary = df_raw.isnull().sum().sort_values(ascending=False)
missing_pct = (missing_summary / len(df_raw) * 100).round(2)

if missing_summary.sum() > 0:
    missing_df = pd.DataFrame({
        'Missing_Count': missing_summary,
        'Missing_Percentage': missing_pct
    })
    missing_df = missing_df[missing_df['Missing_Count'] > 0]
    print(missing_df)
    print(f"\n⚠️ Total missing values: {missing_summary.sum():,}")
else:
    print("✅ No missing values found in the dataset!")


3️⃣ MISSING VALUES SUMMARY:
----------------------------------------
✅ No missing values found in the dataset!


In [28]:
# 4. Duplicate Rows Check
duplicates = df_raw.duplicated().sum()
print(f"\n4️⃣ DUPLICATE ROWS:")
print(f"   • Duplicate rows: {duplicates:,}")
if duplicates > 0:
    print("⚠️ Consider removing duplicates in data cleaning step")
else:
    print("✅ No duplicate rows found")


4️⃣ DUPLICATE ROWS:
   • Duplicate rows: 0
✅ No duplicate rows found


# Cell 3: Statistical summary and categorical field analysis

In [32]:
# Separate numerical and categorical columns
numerical_cols = df_raw.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = ['Month', 'Year', 'Holidays_Count', 'Days']  # These are categorical despite being numeric
pure_numerical_cols = [col for col in numerical_cols if col not in categorical_cols]

print("📊 NUMERICAL VARIABLES STATISTICS:")
print("(Pollution concentrations and AQI values)")
stats_df = df_raw[pure_numerical_cols + ['AQI']].describe()
print(stats_df.round(2).to_string())

📊 NUMERICAL VARIABLES STATISTICS:
(Pollution concentrations and AQI values)
          Date    PM2.5     PM10      NO2      SO2       CO    Ozone      AQI      AQI
count  1461.00  1461.00  1461.00  1461.00  1461.00  1461.00  1461.00  1461.00  1461.00
mean     15.73    90.77   218.22    37.18    20.10     1.03    36.34   202.21   202.21
std       8.80    71.65   129.30    35.23    16.54     0.61    18.95   107.80   107.80
min       1.00     0.05     9.69     2.16     1.21     0.27     2.70    19.00    19.00
25%       8.00    41.28   115.11    17.28     7.71     0.61    24.10   108.00   108.00
50%      16.00    72.06   199.80    30.49    15.43     0.85    32.47   189.00   189.00
75%      23.00   118.50   297.75    45.01    26.62     1.24    45.73   284.00   284.00
max      31.00  1000.00  1000.00   433.98   113.40     4.70   115.87   500.00   500.00


In [33]:
# Date field analysis
print("📅 DATE FIELD:")
print(f"   • Date range: Days 1-31 (represents days of month)")
print(f"   • Unique dates: {df_raw['Date'].nunique()}")

📅 DATE FIELD:
   • Date range: Days 1-31 (represents days of month)
   • Unique dates: 31


In [34]:
# Month distribution
print(f"\n📅 MONTH DISTRIBUTION:")
month_counts = df_raw['Month'].value_counts().sort_index()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for month, count in month_counts.items():
    pct = (count / len(df_raw) * 100)
    print(f"   • {month_names[month-1]:3s} ({month:2d}): {count:3d} records ({pct:5.1f}%)")


📅 MONTH DISTRIBUTION:
   • Jan ( 1): 124 records (  8.5%)
   • Feb ( 2): 113 records (  7.7%)
   • Mar ( 3): 124 records (  8.5%)
   • Apr ( 4): 120 records (  8.2%)
   • May ( 5): 124 records (  8.5%)
   • Jun ( 6): 120 records (  8.2%)
   • Jul ( 7): 124 records (  8.5%)
   • Aug ( 8): 124 records (  8.5%)
   • Sep ( 9): 120 records (  8.2%)
   • Oct (10): 124 records (  8.5%)
   • Nov (11): 120 records (  8.2%)
   • Dec (12): 124 records (  8.5%)


In [35]:
# Year distribution
print(f"\n📅 YEAR DISTRIBUTION:")
year_counts = df_raw['Year'].value_counts().sort_index()
for year, count in year_counts.items():
    pct = (count / len(df_raw) * 100)
    print(f"   • {year}: {count:3d} records ({pct:5.1f}%)")



📅 YEAR DISTRIBUTION:
   • 2021: 365 records ( 25.0%)
   • 2022: 365 records ( 25.0%)
   • 2023: 365 records ( 25.0%)
   • 2024: 366 records ( 25.1%)


In [36]:
# Day of week distribution
print(f"\n📅 DAY OF WEEK DISTRIBUTION:")
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
days_counts = df_raw['Days'].value_counts().sort_index()
for day_num, count in days_counts.items():
    pct = (count / len(df_raw) * 100)
    day_name = day_names[day_num-1] if day_num <= 7 else f"Day_{day_num}"
    print(f"   • {day_name:9s} ({day_num}): {count:3d} records ({pct:5.1f}%)")


📅 DAY OF WEEK DISTRIBUTION:
   • Monday    (1): 209 records ( 14.3%)
   • Tuesday   (2): 209 records ( 14.3%)
   • Wednesday (3): 208 records ( 14.2%)
   • Thursday  (4): 208 records ( 14.2%)
   • Friday    (5): 209 records ( 14.3%)
   • Saturday  (6): 209 records ( 14.3%)
   • Sunday    (7): 209 records ( 14.3%)


In [37]:
# Holidays analysis
print(f"\n🎉 HOLIDAYS ANALYSIS:")
holiday_counts = df_raw['Holidays_Count'].value_counts().sort_index()
for holiday, count in holiday_counts.items():
    pct = (count / len(df_raw) * 100)
    holiday_label = "No Holiday" if holiday == 0 else "Holiday"
    print(f"   • {holiday_label}: {count:4d} records ({pct:5.1f}%)")


🎉 HOLIDAYS ANALYSIS:
   • No Holiday: 1184 records ( 81.0%)
   • Holiday:  277 records ( 19.0%)


# Cell 4: Data quality insights and target variable analysis

In [38]:
# Check for extreme values/outliers
print("🔍 EXTREME VALUES DETECTION:")

pollution_vars = ['PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'Ozone', 'AQI']
for var in pollution_vars:
    Q1 = df_raw[var].quantile(0.25)
    Q3 = df_raw[var].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df_raw[(df_raw[var] < lower_bound) | (df_raw[var] > upper_bound)]
    outlier_pct = (len(outliers) / len(df_raw) * 100)

    print(f"   • {var:6s}: {len(outliers):3d} outliers ({outlier_pct:4.1f}%) | Range: [{df_raw[var].min():6.1f}, {df_raw[var].max():6.1f}]")

🔍 EXTREME VALUES DETECTION:
   • PM2.5 :  65 outliers ( 4.4%) | Range: [   0.1, 1000.0]
   • PM10  :  19 outliers ( 1.3%) | Range: [   9.7, 1000.0]
   • NO2   :  85 outliers ( 5.8%) | Range: [   2.2,  434.0]
   • SO2   :  83 outliers ( 5.7%) | Range: [   1.2,  113.4]
   • CO    :  80 outliers ( 5.5%) | Range: [   0.3,    4.7]
   • Ozone :  57 outliers ( 3.9%) | Range: [   2.7,  115.9]
   • AQI   :   0 outliers ( 0.0%) | Range: [  19.0,  500.0]


In [40]:
# Check for suspicious round numbers or repeated values
print(f"\n🎯 SUSPICIOUS PATTERNS:")
for var in pollution_vars:
    # Check for values at exactly 1000 (possible data cap)
    thousand_values = (df_raw[var] == 1000).sum()
    if thousand_values > 0:
        print(f"   • {var}: {thousand_values} values at exactly 1000 (possible data capping)")

    # Check for values at exactly 500 (AQI max)
    if var == 'AQI':
        five_hundred_values = (df_raw[var] == 500).sum()
        if five_hundred_values > 0:
            print(f"   • {var}: {five_hundred_values} values at exactly 500 (AQI maximum)")


🎯 SUSPICIOUS PATTERNS:
   • PM2.5: 1 values at exactly 1000 (possible data capping)
   • PM10: 1 values at exactly 1000 (possible data capping)
   • AQI: 2 values at exactly 500 (AQI maximum)


In [41]:
# AQI categories according to Indian standards
def categorize_aqi(aqi):
    if aqi <= 50:
        return 'Good'
    elif aqi <= 100:
        return 'Satisfactory'
    elif aqi <= 200:
        return 'Moderate'
    elif aqi <= 300:
        return 'Poor'
    elif aqi <= 400:
        return 'Very Poor'
    else:
        return 'Severe'


In [42]:
# Add AQI categories
df_raw['AQI_Category'] = df_raw['AQI'].apply(categorize_aqi)

print("📊 AQI DISTRIBUTION BY CATEGORIES:")
aqi_dist = df_raw['AQI_Category'].value_counts()
aqi_order = ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe']

for category in aqi_order:
    if category in aqi_dist.index:
        count = aqi_dist[category]
        pct = (count / len(df_raw) * 100)
        print(f"   • {category:12s}: {count:4d} records ({pct:5.1f}%)")


📊 AQI DISTRIBUTION BY CATEGORIES:
   • Good        :   51 records (  3.5%)
   • Satisfactory:  267 records ( 18.3%)
   • Moderate    :  463 records ( 31.7%)
   • Poor        :  384 records ( 26.3%)
   • Very Poor   :  231 records ( 15.8%)
   • Severe      :   65 records (  4.4%)


In [43]:
print(f"\n📈 AQI STATISTICS:")
print(f"   • Mean AQI: {df_raw['AQI'].mean():.1f}")
print(f"   • Median AQI: {df_raw['AQI'].median():.1f}")
print(f"   • Std Dev: {df_raw['AQI'].std():.1f}")
print(f"   • Days with AQI > 300 (Very Poor/Severe): {(df_raw['AQI'] > 300).sum()} ({(df_raw['AQI'] > 300).mean()*100:.1f}%)")
print(f"   • Days with AQI > 200 (Poor+): {(df_raw['AQI'] > 200).sum()} ({(df_raw['AQI'] > 200).mean()*100:.1f}%)")



📈 AQI STATISTICS:
   • Mean AQI: 202.2
   • Median AQI: 189.0
   • Std Dev: 107.8
   • Days with AQI > 300 (Very Poor/Severe): 296 (20.3%)
   • Days with AQI > 200 (Poor+): 680 (46.5%)


# Cell 5: Correlation analysis and final audit summary

In [44]:
# Calculate correlation matrix for pollution variables
pollution_vars = ['PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'Ozone', 'AQI']
corr_matrix = df_raw[pollution_vars].corr()

print("📊 CORRELATION WITH AQI (Target Variable):")
aqi_corr = corr_matrix['AQI'].drop('AQI').sort_values(key=abs, ascending=False)
for var, corr in aqi_corr.items():
    strength = "Very Strong" if abs(corr) >= 0.8 else "Strong" if abs(corr) >= 0.6 else "Moderate" if abs(corr) >= 0.4 else "Weak"
    print(f"   • {var:6s}: {corr:6.3f} ({strength})")


📊 CORRELATION WITH AQI (Target Variable):
   • PM10  :  0.899 (Very Strong)
   • PM2.5 :  0.802 (Very Strong)
   • CO    :  0.697 (Strong)
   • NO2   :  0.319 (Weak)
   • Ozone : -0.164 (Weak)
   • SO2   :  0.036 (Weak)


In [46]:
# Get upper triangle of correlation matrix (excluding diagonal)
import numpy as np
corr_upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
corr_pairs = corr_upper.unstack().dropna().sort_values(key=abs, ascending=False)

for (var1, var2), corr in corr_pairs.head(5).items():
    if var1 != var2:  # Skip self-correlations
        print(f"   • {var1} ↔ {var2}: {corr:.3f}")

   • AQI ↔ PM10: 0.899
   • AQI ↔ PM2.5: 0.802
   • PM10 ↔ PM2.5: 0.722
   • AQI ↔ CO: 0.697
   • CO ↔ PM2.5: 0.690


In [47]:
# Monthly AQI trends
monthly_aqi = df_raw.groupby('Month')['AQI'].agg(['mean', 'std', 'min', 'max']).round(1)
print("📅 AVERAGE AQI BY MONTH:")
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

worst_months = monthly_aqi['mean'].nlargest(3)
best_months = monthly_aqi['mean'].nsmallest(3)

for month, stats in monthly_aqi.iterrows():
    month_name = month_names[month-1]
    marker = "🔴" if month in worst_months.index else "🟢" if month in best_months.index else "  "
    print(f"   {marker} {month_name}: {stats['mean']:6.1f} avg (±{stats['std']:5.1f})")


📅 AVERAGE AQI BY MONTH:
   🔴 Jan:  305.7 avg (± 88.2)
      Feb:  239.7 avg (± 83.4)
      Mar:  200.1 avg (± 55.9)
      Apr:  222.4 avg (± 70.8)
      May:  199.7 avg (± 77.8)
      Jun:  164.0 avg (± 68.6)
   🟢 Jul:   90.4 avg (± 42.8)
   🟢 Aug:   89.8 avg (± 36.1)
   🟢 Sep:   87.3 avg (± 36.5)
      Oct:  191.6 avg (± 90.1)
   🔴 Nov:  342.1 avg (± 72.2)
   🔴 Dec:  297.3 avg (± 84.8)


In [48]:

# Day of week patterns
print(f"\n📅 AVERAGE AQI BY DAY OF WEEK:")
daily_aqi = df_raw.groupby('Days')['AQI'].mean().round(1)
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for day_num, avg_aqi in daily_aqi.items():
    day_name = day_names[day_num-1] if day_num <= 7 else f"Day_{day_num}"
    print(f"   • {day_name:9s}: {avg_aqi:6.1f}")


📅 AVERAGE AQI BY DAY OF WEEK:
   • Monday   :  198.2
   • Tuesday  :  204.4
   • Wednesday:  203.5
   • Thursday :  203.2
   • Friday   :  203.9
   • Saturday :  200.6
   • Sunday   :  201.7


In [49]:
# Holiday vs non-holiday
holiday_aqi = df_raw.groupby('Holidays_Count')['AQI'].agg(['mean', 'count']).round(1)
print(f"\n🎉 AQI: HOLIDAYS vs NON-HOLIDAYS:")
for holiday, stats in holiday_aqi.iterrows():
    holiday_label = "Regular Days" if holiday == 0 else "Holiday Days"
    print(f"   • {holiday_label:12s}: {stats['mean']:6.1f} avg AQI ({stats['count']} days)")



🎉 AQI: HOLIDAYS vs NON-HOLIDAYS:
   • Regular Days:  202.2 avg AQI (1184.0 days)
   • Holiday Days:  202.3 avg AQI (277.0 days)


In [50]:
print("🎯 FINAL AUDIT SUMMARY")
print("="*60)

print(f"✅ DATA QUALITY ASSESSMENT:")
print(f"   • Dataset is CLEAN: No missing values, no duplicates")
print(f"   • Time coverage: 4 years (2021-2024), all months represented")
print(f"   • Balanced temporal distribution across days/months")
print(f"   • Some outliers present (~1-6% per variable) - normal for pollution data")
print(f"   • Potential data capping at 1000 for PM2.5/PM10 (1 case each)")

print(f"\n🎯 TARGET VARIABLE (AQI) INSIGHTS:")
print(f"   • Highly concerning air quality: 46.5% of days are 'Poor' or worse")
print(f"   • Only 21.8% of days have 'Good' or 'Satisfactory' air quality")
print(f"   • Strong correlation with PM2.5 ({aqi_corr['PM2.5']:.3f}) - expected primary driver")

print(f"\n🎯 RECOMMENDED NEXT STEPS:")
print(f"   • Primary target: AQI (well-distributed, no missing values)")
print(f"   • Key predictors: PM2.5, PM10, NO2 (strongest AQI correlations)")
print(f"   • Consider seasonal patterns (monthly variations observed)")
print(f"   • Handle outliers carefully - may represent real extreme pollution events")
print(f"   • Feature engineering: Add seasonal indicators, lagged values, rolling averages")


🎯 FINAL AUDIT SUMMARY
✅ DATA QUALITY ASSESSMENT:
   • Dataset is CLEAN: No missing values, no duplicates
   • Time coverage: 4 years (2021-2024), all months represented
   • Balanced temporal distribution across days/months
   • Some outliers present (~1-6% per variable) - normal for pollution data
   • Potential data capping at 1000 for PM2.5/PM10 (1 case each)

🎯 TARGET VARIABLE (AQI) INSIGHTS:
   • Highly concerning air quality: 46.5% of days are 'Poor' or worse
   • Only 21.8% of days have 'Good' or 'Satisfactory' air quality
   • Strong correlation with PM2.5 (0.802) - expected primary driver

🎯 RECOMMENDED NEXT STEPS:
   • Primary target: AQI (well-distributed, no missing values)
   • Key predictors: PM2.5, PM10, NO2 (strongest AQI correlations)
   • Consider seasonal patterns (monthly variations observed)
   • Handle outliers carefully - may represent real extreme pollution events
   • Feature engineering: Add seasonal indicators, lagged values, rolling averages

✅ Ready to proc